In [1]:
import csv
import numpy as np 
import pandas as pd


from bs4 import BeautifulSoup as bs
import json
import re
import requests
from config import access_token, client_secret, client_id

In [36]:
# lyricsgenius wrapper to get Top 100 Snoop lyrics
import lyricsgenius
genius = lyricsgenius.Genius(access_token, sleep_time=0.01, verbose=False, remove_section_headers = True, excluded_terms = ["(Remix)", "(Live)"], skip_non_songs= True)
artist = genius.search_artist('Snoop Dogg', max_songs = 100, get_full_info= False)
artist.save_lyrics()

In [2]:
with open ('Lyrics_SnoopDogg.json') as json_file:
    data = json.load(json_file)

In [31]:
data['songs'][0]['lyrics']

"(Ugh) Ha-ha-ha, I'm serious, nigga\nOne of y'all niggas got some bad motherfuckin' breath\n(Oh, man) Aye, baby, aye, baby, (shit) aye, baby\nGet some bubblegum in this motherfucker or somethin'\nAye, nigga, get somethin' to eat, dog\nAye, nigga, study long, study wrong, nigga\n\nWith so much drama in the L-B-C\nIt's kind of hard bein' Snoop D-O-double-G\nBut I, somehow, some way\nKeep comin' up with funky-ass shit, like, every single day\nMay I kick a little something for the G's\nAnd make a few ends as I breeze through?\nTwo in the mornin' and the party's still jumpin'\n‘Cause my momma ain't home\nI got bitches in the livin' room gettin' it on\nAnd they ain't leavin' till six in the mornin'\nSo what you wanna do?\nShit, I got a pocket full of rubbers and my homeboys do too\nSo turn off the lights and close the doors\nBut (but what?) we don't love them hoes, yeah\nSo we gon' smoke an ounce to this\nG's up, hoes down, while you motherfuckers bounce to this\nRollin' down the street, smo

In [34]:
# read Json, extract to df 
title = []
url = []
lyrics = []
for i in data['songs']:
    title.append(i['title']),
    url.append(i['url']),
    lyrics.append(i['lyrics'])

snoop_songs = pd.DataFrame(zip(title,url,lyrics),columns= ['Title','URL','Lyrics'])
    

In [35]:
# # drop a row since its a comic not song
# snoop_songs.drop([len(snoop_songs)-2],inplace=True).reset_index(drop=True)


# replace line break, backward slash, random string at the end, texts within parenthesis
repl = {'\n':' ', '\\' : '', 'EmbedShare URLCopyEmbedCopy' : '' }
for key, value in repl.items():
    snoop_songs['Lyrics'] = snoop_songs['Lyrics'].str.replace(key,value)
    
for i in range(len(snoop_songs['Lyrics'])):
    snoop_songs['Lyrics'][i]= re.sub(r"\([^()]*\)", "", snoop_songs['Lyrics'][i])
    # wrapper had an issue, some lyrics have extra 1-2 number digit added to the end. 
    if snoop_songs['Lyrics'][i][-1].isdigit() == True:
        snoop_songs['Lyrics'][i] = snoop_songs['Lyrics'][i][:-1]

        if snoop_songs['Lyrics'][i][-1].isdigit() == True:
            snoop_songs['Lyrics'][i] = snoop_songs['Lyrics'][i][:-1]
        else :
            pass
    
    
snoop_songs.to_csv("Snoop_Lyrics.csv", index=False)